In [11]:
import requests
import pandas as pd
from names_api_evaluator import NamesAPIEvaluator

In [13]:
# Used this blog post: https://juliensalinas.com/en/REST_API_fetching_go_golang_vs_python/
# linked from the API's website: https://www.nameapi.org/en/developer/downloads/

def fetch_from_names_api(name):

    """
    Fetch the NameAPI.org REST API and turn JSON response into Python dict.

    Sent data have to be JSON data encoded into request body.
    Send request headers must be set to 'application/json'.
    """

    # url of the NameAPI.org endpoint:
    url = (
        "http://rc50-api.nameapi.org/rest/v5.0/genderizer/persongenderizer?"
        "apiKey=725a6a1ddf0d0f16f7dc3a6a73a9ac5b-user1"
    )

    # Dict of data to be sent to NameAPI.org:
    payload = {
        "inputPerson": {
            "type": "NaturalInputPerson",
            "personName": {
                "nameFields": [
                    {
                        "string": name,
                        "fieldType": "FULLNAME"
                    }
                ]
            }
        }
    }

    # Proceed, only if no error:
    try:
        # Send request to NameAPI.org by doing the following:
        # - make a POST HTTP request
        # - encode the Python payload dict to JSON
        # - pass the JSON to request body
        # - set header's 'Content-Type' to 'application/json' instead of
        #   default 'multipart/form-data'
        resp = requests.post(url, json=payload)
        resp.raise_for_status()
        # Decode JSON response into a Python dict:
        resp_dict = resp.json()
        return resp_dict
    except requests.exceptions.HTTPError as e:
        print("Bad HTTP status code:", e)
    except requests.exceptions.RequestException as e:
        print("Network error:", e)


### Can it handle surnames?

In [14]:
print(fetch_from_names_api('Samir Amin'))

{'confidence': 0.9115827664399092, 'gender': 'MALE'}


### Double names (where the order matters)

In [18]:
names = ['Hans Joachim', 'Hans-Joachim', 'Maria-José', 'José Maria', 'Jose Maria', 'José-Maria', 'Josémaria', 
         'theo c. m']

In [19]:
for n in names:
    print(fetch_from_names_api(n))

{'gender': 'MALE', 'confidence': 0.8594326867386308}
{'gender': 'MALE', 'confidence': 0.7875648600958193}
{'gender': 'FEMALE', 'confidence': 1.0}
{'gender': 'MALE', 'confidence': 1.0}
{'gender': 'MALE', 'confidence': 0.8960340071451184}
{'gender': 'MALE', 'confidence': 1.0}
{'gender': 'MALE', 'confidence': 1.0}
{'gender': 'MALE', 'confidence': 1.0}


### Names with different gender depending on ethnicity

In [20]:
names = ['Nicola', 'Andrea', 'Alex', 'Mika', 'Addison', 'Ash', 'Dakota']

In [21]:
for n in names:
    print(n), print(fetch_from_names_api(n))

Nicola
{'gender': 'FEMALE', 'confidence': 0.9111111111111111}
Andrea
{'gender': 'MALE', 'confidence': 0.9111111111111111}
Alex
{'gender': 'NEUTRAL', 'confidence': 1.0}
Mika
{'gender': 'NEUTRAL', 'confidence': 1.0}
Addison
{'gender': 'UNKNOWN', 'confidence': 0.8}
Ash
{'gender': 'NEUTRAL', 'confidence': 1.0}
Dakota
{'gender': 'NEUTRAL', 'confidence': 0.911111111111111}


* Nicola and Andrea have much lower confidence
* Neutral names are marked as such

### Check for nonsense words

In [22]:
names = ['the', 'a', 'with', 'an', 'I', 'my']

In [23]:
for n in names:
    print(n), print(fetch_from_names_api(n))

the
{'gender': 'MALE', 'confidence': 0.7872962440437796}
a
{'gender': 'UNKNOWN', 'confidence': 0.6400000000000001}
with
{'gender': 'UNKNOWN', 'confidence': 0.8}
an
{'gender': 'UNKNOWN', 'confidence': 0.8}
I
{'gender': 'UNKNOWN', 'confidence': 0.6400000000000001}
my
{'gender': 'UNKNOWN', 'confidence': 0.8}


Most nonsense words are recognised as such

### Capital letters

In [26]:
names = ['pierre', 'Pierre', 'paul', 'Paul']

In [27]:
for n in names:
    print(n), print(fetch_from_names_api(n))

pierre
{'gender': 'MALE', 'confidence': 0.8975930513502768}
Pierre
{'gender': 'MALE', 'confidence': 0.8975876602995245}
paul
{'gender': 'MALE', 'confidence': 0.9111111111111112}
Paul
{'gender': 'MALE', 'confidence': 0.9111111111111111}


**For names such as 'paul pinsky' and 'paul horwich' the API returns 'f'.
If both are written in capital letters then it returns 'm' (also first name alone is considered 'm' by the API):**

In [28]:
print(fetch_from_names_api('Paul horwich'))

{'gender': 'MALE', 'confidence': 0.9111111111111111}


In [29]:
print(fetch_from_names_api('Paul ribenboim'))

{'gender': 'MALE', 'confidence': 0.9111111111111111}


In [30]:
print(fetch_from_names_api('Paul pinsky'))

{'gender': 'MALE', 'confidence': 0.9111111111111111}
